# needed

# database with college football teams, colors, logos, game info (https://collegefootballdata.com/exporter)

# open ai key so I can use their image generation (https://www.geeksforgeeks.org/generate-images-with-openai-in-python/)

# s3 so I can save designs

 


In [1]:

from PIL import Image, ImageOps, ImageDraw, ImageFont
import numpy as np
import warnings
import requests
import pandas as pd
import json
import datetime as dt
import cfbd
import helpers as hf
warnings.filterwarnings("ignore", category=DeprecationWarning) 
import yaml
import random
import base64
import svgwrite
from io import BytesIO

In [2]:
# head = {
# "Authorization": 'Bearer',
# "Content-Type": "application/json"
# }

# url = "https://api.printify.com/v1/shops/13647323/products/65a37b41feca5ad9cb0c1a22.json"

# resp = requests.get(url = url, headers=head)


In [3]:
# json.loads(resp.text)

In [4]:
def process_game(game,teams,design_config,s3_mover,game_id,test,shop_config,pref):
    print(f'    generating game image')
    game = game.to_dict('records')[0]
    game_config = hf.parse_game(game,teams)
    game_config['game_id'] = game
    config = hf.combine_configs(design_config,game_config)
    design,text = hf.build_design(config,test=False)

    # do I need to save all my images to s3?
    s3_mover.image_to_s3(design,rf"{pref}/{game_config['home_team']['shortn']}_v_{game_config['away_team']['shortn']}_{game_id}")

    if not test:
        title,description,tags=hf.generate_t_d_t(game_config)
        shop_config['image'] = design
        shop_config['title'] = title
        shop_config['description'] =  description
        shop_config['tags'] =  tags
        shop_config['design'] = design
        shop_config['text'] = text
        hf.post(shop_config,publish=True)

In [ ]:
import time

In [5]:
def monday_run(test=False,specific_game = None):
    design_config,shop_config = hf.get_config()
    s3_mover = hf.s3_mover()
    cfbd_loader = hf.cfbp_handler(design_config['cfbd_api'],'09-6-2023')
    yr = dt.date.today().year
    yr=2023

    # refresh teams if new season, could also add something to check if any teams exist
    if hf.new_football_season():
        teams = cfbd_loader.get_team_info()
        s3_mover.pd_to_s3(teams,f'teams/all_teams_{yr}')
    else:
        # get my df of teams
        teams = s3_mover.s3_to_pd(f'teams/all_teams_{yr}')
    
    # get new schedule info and return list of games i need to do
    all_games = cfbd_loader.get_schedule(yr)
    old_games = s3_mover.s3_to_pd(s3_name=f'old/all_games_{yr}')
    to_do_new,to_do_upcoming = cfbd_loader.determine_to_do(old_games)

    if specific_game is not None:
        to_do_new = []
        to_do_upcoming = [specific_game]

    print(f'New games to do {len(to_do_new)}')
    print(f'Upcoming games to do {len(to_do_upcoming)}')

    to_do_upcoming = to_do_upcoming[:10]

    # have to line image path for config up with
    if len(to_do_new)>1:
        print('new games')
        game_weeks = []
        for game_id in to_do_new:
            game = all_games.loc[all_games.id == game_id]
            if game.game_type.values[0] == 'regular':
                game_weeks.append(game.week.values[0])
            else: 
                wk = game.game_type.values[0]+str(game.week.values[0])
                game_weeks.append(wk)
            process_game(game,teams,design_config,s3_mover,game_id,test,shop_config,pref='post')
        week = hf.most_frequent(game_weeks)

    if len(to_do_upcoming):
        print('upcoming games')
        game_weeks = []
        for game_id in to_do_upcoming:
            game = all_games.loc[all_games.id == game_id]
            if game.game_type.values[0] == 'regular':
                game_weeks.append(game.week.values[0])
            else: 
                wk = game.game_type.values[0]+''+str(game.week.values[0])
                game_weeks.append(wk)
            process_game(game,teams,design_config,s3_mover,game_id,test,shop_config,pref='pre')
        week_upcoming = hf.most_frequent(game_weeks)
    
    time.sleep(10*60)
    hf.create_collection(to_do_new,week,yr,shop_config,pref='post')
    hf.create_collection(to_do_upcoming,week_upcoming,yr,shop_config,pref='pre')
    s3_mover.pd_to_s3(all_games,s3_name=f'old/all_games_{yr}')

In [9]:
monday_run(test=False,specific_game=401551788)

artificially setting to monday past date
New games to do 0
Upcoming games to do 1
upcoming games
    generating game image
NCAA football University Of Washington Huskies mascot beating University Of Texas Longhorns mascot
using saved image because test mode
image and text sent to printify
Product posted successfully in Printify
Product published successfully in Printify
['postseason1']
got all items
filtered list for just this collection
collection created


- handle nan in teams table
- font auto sizing

In [7]:
test = pd.read_csv(r'/Users/loganmckerlich/Downloads/all_games_2023 __.csv',index_col=0)